# =======단일 키워드 검색용 로직=======

### 세션만료까지 걸리는시간 테스트 : 1125부) 65분까지 OK.

## 크롤링 정책 : 
request 별로 랜덤하게 sleep, 키워드 당 1분 정도로 천천히
계정 별로 다른 IP와 user-agent사용 (꾸준히, 로그인 인증되어야함) - 사람으로 인식되도록
주기적으로 재 로그인
작업은 최대 10분을 넘기지 않기

## 예외처리
각종 예외상황(검열, 로그인 checkpoint 필요,, 등)에서 서버단에서 어떻게 대처하고 (세션 교체 등)
클라이언트단에 어떻게 return할지 설계해놓아야함. 
=> 추가구현 : 이상 발생시 재로그인 후 다시 이상 발생하면 usable_acc에서 지우기 (usable_acc 관리하는 로직) / 계정정보도 DB로 저장해야할듯?

In [31]:
from requests import get, post, Session
from urllib import parse
import json
from datetime import datetime, timedelta
import emoji
import time
import pymysql
import random
from db import *

## DB 연결 정보

In [2]:
host = 'hotkey.czighwjgpkkz.ap-northeast-2.rds.amazonaws.com' 
user = 'admin'
pw = 'hotkey123'
port = 3306

## Account 정보 & cur_idx => 현재 이용하고 있는 계정

In [3]:
# 전체 계정 (사실상 전부 차단 전력있음)
total_acc = [('kj10522002@korea.ac.kr','kj76081460!'), ('quickpass8@gmail.com','thskrl1!'),('hotkey2','gktzl2'),
                 ('seungirumd+1@gmail.com', 'pinstaw25'), ('lshyun0510.11@gmail.com', '*thvmxmdnpdj'),
                 ('seyun1052dev@gmail.com', 'kj76081460!'), ('lshyun0510.12@gmail.com', '*thvmxmdnpdj'),
                 ('quickpass88@gmail.com', 'thskrl1!'), ('hotkey22', 'gktzl22!'), ('tamikia0@hbviralbv.com', 'cvd0^ktm')]

# 사용중인 계정 리스트
using_acc = [(i[0], i[1], datetime.now()) for i in total_acc]
# 임시 테스트용..
using_acc = [('kj10522002@korea.ac.kr','kj76081460!'), ('tamikia0@hbviralbv.com','cvd0^ktm')]

In [4]:
using_acc.sort(key=lambda x:x[2]) #최종 로그인 날짜에 따라 sorting하는 logic

IndexError: tuple index out of range

### 그 외 초기값..

In [5]:
delimiter = 'HOTKEY123!@#'
cur_idx = -1

## 로그인 함수

In [11]:
def login(id, pw):
    # 새로운 헤더, 세션 생성 (새로운 세션 리턴)
    # 헤더, 세션, status를 리턴
    # status = 0 : success , 1 : checkpoint_needed 에러 (수동 인증 필요) 2 : incorrect pw 에러 (임시 차단) 3. CSRF 에러 (IP문제) 4. 그 외

    header, session = dict(), Session()
    header['user-agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.56'
    header['x-ig-app-id'] = '936619743392459'

    resp = session.get('https://www.instagram.com/data/manifest.json', headers=header)
    header['x-csrftoken'] = session.cookies.get('csrftoken')
    url = 'https://i.instagram.com/api/v1/web/accounts/login/ajax/'
    params = {'enc_password': '#PWD_INSTAGRAM_BROWSER:0:{}:{}'.format(
        int(datetime.now().timestamp()), pw), 'username': id, 'optIntoOneTap': 'false'}

    resp = session.post(url, data=params, headers=header)  # 로그인 요청

    if resp.status_code == 400:
        print('Login Error : checkpoint_needed (수동 인증 필요) ...')
        print(resp.text)
        return header, session, 1
    elif resp.status_code == 403:
        print('Login Error : CSRF error (IP 문제) ,,, ')
        print(resp.text)
        return header, session, 3
    elif resp.status_code == 200:
        if not (resp.json()['authenticated']):
            print('Login Error : incorrect pw (계정 임시 차단 혹은 pw인증 에러)...')
            print(resp.json())
            return header,session,2
        else:
            print('Login Successful')
            return header, session, 0
    else:
        print('Login Error : unknown,,, response status code:', resp.status_code)
        print(resp.text)
        return header, session, 4

## 세션을 한번에 생성하는 함수 & 존재하는 세션 갱신하는 함수

In [8]:
def gen_sessions(acc_list):
    # id 별로 세션 생성 (로그인) ....
    headers = list()
    sessions = list()
    times = list()
    use_acc = list()
    
    for acc in acc_list:
        session, header = Session(), dict()
        #header['user-agent'] = h.generate()['User-Agent'] #요청시마다 다른 user-agent 뽑아줌
        header['user-agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.56'
        header['x-ig-app-id'] = '936619743392459'
        header, session, login_success = login(acc[0], acc[1], header, session)
        if login_success :
            print('successfully logged in...', acc, '\n')
            headers.append(header)
            sessions.append(session)
            use_acc.append(acc)
            times.append(datetime.now())
        else:
            print('login_unsuccessful : ', acc,'\n')
        time.sleep(random.uniform(1,5))
    return (sessions, headers, use_acc, times) #최종 로그인한 시간

def new_login(sessions, headers, use_acc, times): 
    # 전체 훑으면서 갱신 (defaeult)
    for i in range(len(sessions)):
        if datetime.now() - times[i] > timedelta(hours=1): #7~10분 사이의 랜덤한 시간이상 지났으면
        #if datetime.now() - times[i] > timedelta(seconds=5): #실험용
            session, header = Session(), dict()
            #header['user-agent'] = h.generate()['User-Agent']
    
            header['user-agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.56'
            header['x-ig-app-id'] = '936619743392459'
            #해당 계정들에 대해 재 로그인 시도
            header, session, login_success = login(use_acc[i][0], use_acc[i][1], header, session)
            if login_success : #로그인 성공
                print('re-login complete...', use_acc[i], '\n')
                headers[i], sessions[i], times[i] = header, session, datetime.now()
            else: #로그인 실패
                print('re-login_unsuccessful : ', use_acc[i], '\n')
                headers.pop(i),
                sessions.pop(i), 
                times.pop(i), 
                use_acc.pop(i)
            time.sleep(random.uniform(1,5))
    return (sessions, headers, use_acc, times)

In [28]:
header, session, status = login('tamikia0@hbviralbv.com', 'cvd0^ktm')

Login Successful


In [29]:
hot_key_instagram_recent('치킨', header, session)

누적 게시물 수 : 27
누적 게시물 수 : 54
누적 게시물 수 : 84
누적 게시물 수 : 102
누적 게시물 수 : 132
누적 게시물 수 : 159
누적 게시물 수 : 189
누적 게시물 수 : 216
누적 게시물 수 : 246
누적 게시물 수 : 264
누적 게시물 수 : 291
누적 게시물 수 : 300
최근 게시물 수집 완료!
총 소요시간 :  0:01:05.487501 




(True,
 [['Posted @withrepost • @sarafriends_official 안녕, 나는 사라 :fox:\n\n사라프렌즈 캘린더에 옥의 티가 있다 :double_exclamation_mark:\n\n옥의 티를 찾은 사람에게는 선물을 증정하겠어 0_:sparkles:\n\n#참여방법\n:red_heart:@sarafriends_official 팔로우하기\n:orange_heart:사라프렌즈 캘린더에서 옥의 티를 찾은 후 댓글을 남기기\n:yellow_heart:게시물 리그램하면 완료 !! \n-\n#댓글로_친구태그시_당첨확률_UP\n-\n:check_mark:이벤트 기간: 11/23(수)~11/30 (수)\n:check_mark:당첨자 발표: 12/1 (목)\n:check_mark:당첨 선물: 치킨 (3명) , 스타벅스 커피쿠폰 (5명)\n\n#리그램 #리그램이벤트 #리그램진행중 #리그램이벤트중 #네일그램 #이벤트 #이벤트참여 #선팔 #맞팔 #행사중 #일상 #일상소통 #오픈 #오픈이벤트 #소통 #일상스타그램 #치킨 #스타벅스 #기프티콘',
   "유튜브 구독 이벤트!\n\n:bear:이벤트 참여방법\n경남문화예술진흥원 유튜브 채널 구독, 좋아요 캡처> 네이버폼을 통한 접수\n\n:bear:네이버 폼 링크\nhttps://naver.me/x7r86Nns\n\n:bear:이벤트 기간: '22.11.28.(월)~12.12.(월)\n:bear:경품: 교촌치킨 기프티콘\n\n진흥원 유튜브 구독! 능소니 인스타는 팔로우! 어서 참여해주세요~!!! \n\n#치킨 #치킨이벤트 #이벤트 #구독이벤트 #유튜브구독 #유튜브 #유튜브구독이벤트 #인스타그램이벤트 #팔로우이벤트 #교촌치킨 #교촌치킨이벤트 #치킨드립니다 #경남문화예술진흥원 #경남 #문화 #예술 #능소니 #아기곰 #곰돌이 #귀여워 #선팔하면맞팔 #팔로우 #팔로우반사",
   ':soccer_ball:\n축구 응원봉은 치킨과 피자로!:poultry_leg::pizza:\n인기 브랜드 배민 앱 최대 7천원 

## 최근 게시물 수집 함수 (최대 300 posts) 

In [25]:
def hot_key_instagram_recent(query, headers, session, max_page=40): 
    #return (status, recent_list, timestamp, image_list)
    #status = 0 : success, 1 : 크롤링 중 문제 발생 (CSRF error), code는 200 , 2 : 400error - 계정에 checkpoint 필요
    # 3 : 검열 혹은 페이지 x,  4 : 그 외
    postcnt = 0 #포스트 수 카운팅, 300개 되면 stop.
    timestamp = 0
    
    before = datetime.now()
    recent_list = list()
    image_list = list()
    
    url = 'https://i.instagram.com/api/v1/tags/web_info/?tag_name={}'.format(parse.quote(query))
    headers['x-csrftoken'] = session.cookies.get('csrftoken') #재설정
    headers['referer'] = 'https://www.instagram.com/explore/tags/{}/'.format(parse.quote(query))
    resp = session.get(url, headers = headers)
    
    if(resp.status_code == 404):
        print('Crawling error : 검열되는 해시태그이거나 해당 태그를 담은 페이지가 존재하지 않습니다.')
        print('response code : ', 404)
        return (3, recent_list, timestamp, image_list)   
    
    elif(resp.status_code != 200): #응답 에러
        if (resp.status_code == 400):
            print('Crawling error : checkpoint needed 의심')
            print(resp.text)
            return (2, recent_list, timestamp, image_list)
        print('Crawling error : unknown..., resp.code :', resp.status_code)
        print(resp.text)
        return (4, recent_list, timestamp, image_list) #계정바꿔서 재시도

    
    #첫페이지 응답 정상인경우
    
    ################################################# 비정상입력 예외처리
    try:
        resource = resp.json()
    except:
        print("Crawling error : 응답 코드는 정상이나, 비정상적인 데이터 형식 반환됨") #크롤링 밴 의심..
        #print(resp.content)
        return (1, recent_list, timestamp, image_list)
    #################################################
    
    #recent 돌기
    
    #print('\n최근 게시물: 1페이지....')
    recent_list.append([])
    image_list.append([])
    for i in resource['data']['recent']['sections']: #0~8
        for k in i['layout_content']['medias']:
            if (k['media']['caption'] != None):
                if 'text' in k['media']['caption'].keys():
                    recent_list[0].append(emoji.demojize(k['media']['caption']['text'])) # 이모티콘 제거 ###########################
                    timestamp = max(timestamp, k['media']['caption']['created_at_utc']) # 최신 포스트의 현재 시간 1669098438 ##############################
                    ###--------------------------1123 추가
                    #######################################################################
                    try:
                        image_list[0].append(k['media']['image_versions2']['candidates'][0]['url']) # 포스트의 이미지 URL 
                    except:
                        image_list[0].append(k['media']['carousel_media'][0]['image_versions2']['candidates'][0]['url'])
                        # 복수의 이미지가 있는 포스트의 첫이미지 URL 
                    #########################################################
                    postcnt+=1
    print('누적 게시물 수 :', postcnt)
    time.sleep(random.uniform(1,5))
                    
    #recent기준 다음 페이지 관련 정보
    recent_info = {'max_id':'', 'page': '', 'isnext': False}
    if resource['data']['recent']['more_available']:
        recent_info['max_id'], recent_info['page'], recent_info['isnext'] = \
        resource['data']['recent']['next_max_id'], resource['data']['recent']['next_page'], True
        
    #그 다음 페이지부터 recent 돌기
    url = 'https://i.instagram.com/api/v1/tags/{}/sections/'.format(parse.quote(query))
    data = {'max_id': recent_info['max_id'], 'page': recent_info['page'], 'surface' : 'grid', 'tab': 'recent'}
    
    while (recent_info['isnext']==True and data['page']<max_page):
        #print('\n최근 게시물: {}페이지....'.format(data['page']+1))
        headers['x-csrftoken'] = session.cookies['csrftoken'] #매번 재설정해주기
        resp = session.post(url, data=data, headers=headers)
        ########################################################################### 스크롤 시작 이후 로그인 소요 발생 시 예외 처리
        if(resp.status_code != 200): #응답 에러
            if (resp.status_code == 400):
                print('Crawling error : checkpoint needed 의심')
                print(resp.text)
                return (2, recent_list, timestamp, image_list)
            print('Crawling error : unknown..., resp.code :', resp.status_code)
            print(resp.text)
            return (4, recent_list, timestamp, image_list) #계정바꿔서 재시도
        ##########################################################################
        ################################################# 비정상입력 예외처리
        try:
            resource = resp.json()
        except:
            print("Crawling error : 응답 코드는 정상이나, 비정상적인 데이터 형식 반환됨") #크롤링 밴 의심..
            #print(resp.content)
            return (1, recent_list, timestamp, image_list)
        #################################################
        recent_list.append([])
        image_list.append([])
        for i in resource['sections']: #0~8
            for k in i['layout_content']['medias']: #0~2
                if (k['media']['caption'] != None):
                    if 'text' in k['media']['caption'].keys():
                        recent_list[data['page']].append(emoji.demojize(k['media']['caption']['text'])) # 이모티콘 제거 ###############
                        ###--------------------------1123 추가
                        #######################################################################
                        try:
                            image_list[data['page']].append(k['media']['image_versions2']['candidates'][0]['url']) # 포스트의 이미지 URL 
                        except:
                            image_list[data['page']].append(k['media']['carousel_media'][0]['image_versions2']['candidates'][0]['url'])
                            # 복수의 이미지가 있는 포스트의 첫이미지 URL 
                        #########################################################
                        postcnt+=1
                        if (postcnt >= 300):
                            print('누적 게시물 수 :', postcnt)
                            print('최근 게시물 수집 완료!')
                            print('총 소요시간 : ', datetime.now() - before,'\n\n')
                            return (0, recent_list, timestamp, image_list)
            
        if resource['more_available']:
            data['max_id'] = resource['next_max_id']
            data['page'] = resource['next_page']
    
        else:
            recent_info['isnext']=False
        print('누적 게시물 수 :', postcnt)
        time.sleep(random.uniform(1,5))
    
    print('누적 게시물 수 :', postcnt)
    print('최근 게시물 수집 완료!')
    print('총 소요시간 : ', datetime.now() - before,'\n')
    return (0, recent_list, timestamp, image_list)

## keyword와 사용가능한 account 정보를 받아 성공여부, corpus와 image들 반환하는 함수

#### 단, 본 로직처럼 요청이 들어올때마다 로그인하는 것은 비합립적..
#### 실제 서비스가 구현이 되고, 요청이 꽤 자주 들어온다면, 주기적으로(1시간-1시간 반) 로그인된 세션을 준비시켜놓고 그것으로 로그인하는것이 바람직할듯 #### (usable acc을 3개정도씩 준비?)

#### 현재는 usable_acc을 준비해서 넣어주겠지만, 나중에는 total_acc > usable_acc > 여러개의 acc_list로 나눠서 사용. 
#### ****cur-idx에 대한 로직 재정립이 필요!!!

#### 혹은 session들을 주기적으로 로그인 시킨 다음 넘겨줄수도 있음.
#### 요청이 들어올때마다 session로그인시키고 로그인된 session정보를 저장(최종 로그인 시간과 함께...)

#### session이 한번 로그인되면 얼마나 유지되는지 실험이 필요 (1시간 이상 지속 확인됨)

In [33]:
def single_search(keyword, acc_list): #성공여부, corpus랑 image를 반환
    # 계정 정보 받아오기
    global cur_idx
    global delimiter
    
    #db연결
    conn, cur = access_db()
    cur.execute('SELECT tid, ttable FROM is_tag NATURAL JOIN tag_info WHERE tname = (%s);', (keyword))
    res = cur.fetchall()
    
    #DB에 존재할 경우
    if len(res)>=1:
        tid, ttable = res[0]['tid'], res[0]['ttable']
        corpus, image = '', ''
        if ttable==1: #s_corpus
            cur.execute('select corpus from s_corpus where tid = (%s)', (tid))
            corpus = cur.fetchall()[0]['corpus']
        elif ttable==2: #t_corpus
            cur.execute('select corpus from t_corpus where tid = (%s)', (tid))
            corpus = cur.fetchall()[0]['corpus']
        elif ttable==3: #n_corpus
            cur.execute('select corpus from n_corpus where tid = (%s)', (tid))
            corpus = cur.fetchall()[0]['corpus']
        else:
            #예외
            print('db 접근 중 발생한 에러입니다. 재시도요망')
            conn.close()
            return (False, '', '')
        cur.execute('select image from images where tid = (%s)', (tid))
        image = cur.fetchall()[0]['image']
        conn.close()
        return (True, corpus, image)
    
    #DB에 존재하지 않을경우, 크롤링->DB적재->값 반환
    #초기값 설정
    total_success = False
    totaltrycnt = 0
    bf = datetime.now() #수행시간 기록용
    
    total_success = False
    recent_list = list()
    image_list = list()
    while (not total_success) and (totaltrycnt < len(acc_list)): #최대 try cnt는 정책따라서 바꾸기
        recent_list = []
        cur_idx = (cur_idx+1)%len(acc_list)
        print('Currently trying with ... :', acc_list[cur_idx][0], acc_list[cur_idx][1]) #현재 로그인 시도 중인 계정
        session, header = Session(), dict()
        #user-agent 바꿔야할거임
        header['user-agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.56'
        header['x-ig-app-id'] = '936619743392459'
        header, session, login_success = login(acc_list[cur_idx][0], acc_list[cur_idx][1], header, session)
        if not login_success :
            totaltrycnt+=1
            continue
        print('successfully logged in...', acc_list[cur_idx][0], acc_list[cur_idx][1], '\n')
        login_success = False
        #수집 시작
        print('수집 시작... keyword :', keyword)
        total_success, recent_list, timestamp, image_list = hot_key_instagram_recent(keyword, header, session)
        if total_success: #수집 성공
            #db에 적재
            corpus = ''
            for page in recent_list:
                for post in page:
                    corpus = corpus+delimiter+post
            cur.execute('insert into is_tag (tname) values (%s);', keyword)
            cur.execute('SELECT tid FROM is_tag ORDER BY tid DESC LIMIT 1;')
            tid = cur.fetchone()['tid']
            #n_corpus에 저장
            cur.execute('insert into tag_info (tid, ttable, up_date, time_stamp) values (%s, %s, %s, %s);', (tid, 3, time.strftime('%Y-%m-%d'), timestamp))
            cur.execute('insert into n_corpus (tid, corpus) values (%s, %s);', (tid, corpus[12:]))
            #image추가하는 로직
            image = ''
            for page in image_list:
                for post in page:
                    image = image+delimiter+post
            cur.execute('insert into images (tid, image) values (%s, %s);', (tid, image[12:]))
            conn.commit()
            conn.close()
            ############(corpus, iamge들 전달)#############
            return (True, corpus[12:], image[12:])
        time.sleep(random.uniform(5,10))
        totaltrycnt+=1
    
    close_db(conn)
    
    return (False, '', '')

In [10]:
result = single_search('hello', using_acc) #검열되는 언어의 경우 False를 리턴

Currently trying with ... : kj10522002@korea.ac.kr kj76081460!
login response error,, response status code: 403
{"message":"CSRF token missing or incorrect","status":"fail"}
Currently trying with ... : tamikia0@hbviralbv.com cvd0^ktm
login response error,, response status code: 403
{"message":"CSRF token missing or incorrect","status":"fail"}


# 1125update : 차단 전력이 있는 ip가 아니라면, 키워드 하나씩 적재하는 것은 잘 작동하는 것으로 보임. (계정이 checkpoint나 임시차단 상태가 아니라면)

# 비고 (추가구현)
### 11.25기준) totaltrycnt를 늘리는 것보다, 로그인 실패와 검색 실패를 나누는 로직이 필요해 보임.
## case별로 다르게 예외처리를 두어야함.

# 추가) 예전에 생성해둔 인기 게시물 수집하는 로직... (11.02 생성)
### 실사용하려면 위 최근 게시물 로직과 같이 예외처리하고 바꿔야함.

In [8]:
def hot_key_instagram_top(query, headers, session, max_page=5):
    before = datetime.now()
    top_list = [[] for i in range(max_page)]
    
    url = 'https://i.instagram.com/api/v1/tags/web_info/?tag_name={}'.format(parse.quote(query))
    headers['x-csrftoken'] = session.cookies.get('csrftoken') #재설정
    resp = session.get(url, headers = headers)
    if(resp.status_code != 200): #응답 에러
        print('response error during crawling : please retry with some other way\n')
        print(resp.text)
        return (False, top_list) #계정바꿔서 재시도
    
    ################################################# 비정상입력 예외처리
    try:
        resource = resp.json()
    except:
        print("비정상적인 키워드 입력")
        return (True, recent_list)
    #################################################
    
    #top 게시물 돌기
    print('인기 게시물: 1페이지....')
    for i in resource['data']['top']['sections']: #0~8
        for k in i['layout_content']['medias']:
            if (k['media']['caption'] != None):
                if 'text' in k['media']['caption'].keys():
                    top_list[0].append(emoji.demojize(k['media']['caption']['text'])) # 이모티콘 제거 ############################
                    print(k['media']['caption']['created_at_utc']) # 포스트의 현재 시간 1669098438 ##############################
                    ###--------------------------1123 추가
                    #######################################################################
                    try:
                        print(k['media']['image_versions2']['candidates'][0]['url']) # 포스트의 이미지 URL 
                    except:
                        print(k['media']['carousel_media'][0]['image_versions2']['candidates'][0]['url'])
                        # 복수의 이미지가 있는 포스트의 첫이미지 URL 
                    #########################################################
                    
    #top기준 다음 페이지 관련 정보
    top_info = {'top_max_id':'', 'top_page': '', 'top_isnext': False}
    if resource['data']['top']['more_available']:
        top_info['top_max_id'], top_info['top_page'], top_info['top_isnext'] = \
        resource['data']['top']['next_max_id'], resource['data']['top']['next_page'], True
    
    #그 다음 페이지부터 돌기
    url = 'https://i.instagram.com/api/v1/tags/{}/sections/'.format(parse.quote(query))
    data = {'max_id': top_info['top_max_id'], 'page': top_info['top_page'], 'surface' : 'grid', 'tab': 'top'} #tab을 top으로 변경
    
    while (top_info['top_isnext']==True and data['page']<max_page):
        headers['x-csrftoken'] = session.cookies['csrftoken'] #매번 재설정해주기
        resp = session.post(url, data=data, headers=headers)
        ########################################################################### 스크롤 시작 이후 로그인 소요 발생 시 예외 처리
        if(resp.status_code != 200): #응답 에러
            print('response error during crawling : please retry with some other way\n')
            print(resp.text)
            return (False, recent_list) #계정바꿔서 재시도
        ##########################################################################
        resource = resp.json()
        print('인기 게시물: {}페이지....'.format(data['page']+1))
        for i in resource['sections']: #0~8
            for k in i['layout_content']['medias']: #0~2
                if (k['media']['caption'] != None):
                    if 'text' in k['media']['caption'].keys():
                        top_list[data['page']].append(emoji.demojize(k['media']['caption']['text'])) # 이모티콘 제거 #################
                        print(k['media']['caption']['created_at_utc']) # 포스트의 현재 시간 1669098438 ##############################\
                        ###--------------------------1123 추가
                        #######################################################################
                        try:
                            print(k['media']['image_versions2']['candidates'][0]['url']) # 포스트의 이미지 URL 
                        except:
                            print(k['media']['carousel_media'][0]['image_versions2']['candidates'][0]['url'])
                            # 복수의 이미지가 있는 포스트의 첫이미지 URL 
                        #########################################################
            
        if resource['more_available']:
            data['max_id'] = resource['next_max_id']
            data['page'] = resource['next_page']
    
        else:
            top_info['top_isnext']=False
    
    print('인기 게시물 수집 완료!\n')
    print('총 소요시간 : ', datetime.now() - before,'\n\n')
    return (True, top_list)